In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,f1_score
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_user_data_train = pd.read_csv("train/user_data.csv")
df_train_submission = pd.read_csv("train/train_submissions.csv")
df_problem_data_train = pd.read_csv("train/problem_data.csv")
df_sample_sub = pd.read_csv("sample_submissions_wbscxqU.csv")
df_test_sub = pd.read_csv("test_submissions_NeDLEvX.csv")

In [3]:
df_problem_data_train.drop(['tags'],axis=1,inplace=True)
#df_problem_data_train.points.fillna(0,inplace=True)
df_problem_data_train.level_type.fillna('N',inplace=True)
df_user_data_train.country.fillna('Missing',inplace=True)

def postive_only(x):
    if(x<0):
        return(-1*x)
    else:
        return(x)
def get_points(x1):
    if(x1=="A"):
        return(500)
    elif(x1=='B'):
        return(1000)
    elif(x1=='C'):
        return(1500)
    elif(x1=='D'):
        return(2000)
    elif(x1=='E'):
        return(2500)
    elif(x1=='F'):
        return(3000)
    elif(x1=='G'):
        return(3500)
    elif(x1=='H'):
        return(4000)
    else:
        return(1000)

#df_problem_data_train['points']=df_problem_data_train['level_type'].map(lambda x: get_points(x))
 df_problem_data_train.drop()   
df_user_data_train['submission_count']=df_user_data_train['submission_count'].map(lambda x: postive_only(x))
df_user_data_train['problem_solved']=df_user_data_train['problem_solved'].map(lambda x: postive_only(x))
df_user_data_train['contribution']=df_user_data_train['contribution'].map(lambda x: postive_only(x))
df_user_data_train['follower_count']=df_user_data_train['follower_count'].map(lambda x: postive_only(x))
df_user_data_train['last_online_time_seconds']=df_user_data_train['last_online_time_seconds'].map(lambda x: postive_only(x))
df_user_data_train['max_rating']=df_user_data_train['max_rating'].map(lambda x: postive_only(x))
df_user_data_train['registration_time_seconds']=df_user_data_train['registration_time_seconds'].map(lambda x: postive_only(x))

#from sklearn.preprocessing import LabelEncoder
#lb = LabelEncoder()
#df_user_data_train['country'] = lb.fit_transform(df_user_data_train['country'])
#df_user_data_train['rank'] = lb.fit_transform(df_user_data_train['rank'])
#df_problem_data_train['level_type'] = lb.fit_transform(df_problem_data_train['level_type'])


In [4]:
df_user_data_train.head()

,user_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_3311,47,40,0,Missing,4,1504111645,348.337,330.849,intermediate,1466686436
1,user_3028,63,52,0,India,17,1498998165,405.677,339.450,intermediate,1441893325
2,user_2268,226,203,8,Egypt,24,1505566052,307.339,284.404,beginner,1454267603
3,user_480,611,490,1,Ukraine,94,1505257499,525.803,471.330,advanced,1350720417
4,user_650,504,479,12,Russia,4,1496613433,548.739,486.525,advanced,1395560498


In [5]:
df_user_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 11 columns):
user_id                      3571 non-null object
submission_count             3571 non-null int64
problem_solved               3571 non-null int64
contribution                 3571 non-null int64
country                      3571 non-null object
follower_count               3571 non-null int64
last_online_time_seconds     3571 non-null int64
max_rating                   3571 non-null float64
rating                       3571 non-null float64
rank                         3571 non-null object
registration_time_seconds    3571 non-null int64
dtypes: float64(2), int64(6), object(3)
memory usage: 307.0+ KB


In [6]:
df_user_data_train['problem_sub_ratio'] = df_user_data_train['problem_solved'] / df_user_data_train['submission_count']

df_user_data_train['last_online_time_seconds'] = df_user_data_train.last_online_time_seconds.max() - df_user_data_train['last_online_time_seconds']
df_user_data_train['last_online_time_minutes'] = df_user_data_train['last_online_time_seconds'] / 60
df_user_data_train['last_online_time_hours'] = df_user_data_train['last_online_time_seconds'] / (60*60)
df_user_data_train['last_online_time_days'] = df_user_data_train['last_online_time_seconds'] / (60*60*24)
df_user_data_train['last_online_time_months'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*30)
df_user_data_train['last_online_time_years'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*365)

df_user_data_train['registration_time_seconds'] = df_user_data_train.registration_time_seconds.max() - df_user_data_train['registration_time_seconds']
df_user_data_train['registration_time_minutes'] = df_user_data_train['registration_time_seconds'] / 60
df_user_data_train['registration_time_hours'] = df_user_data_train['registration_time_seconds'] / (60*60)
df_user_data_train['registration_time_days'] = df_user_data_train['registration_time_seconds'] / (60*60*24)
df_user_data_train['registration_time_months'] = df_user_data_train['registration_time_seconds'] / (60*60*24*30)
df_user_data_train['registration_time_years'] = df_user_data_train['registration_time_seconds'] / (60*60*24*365)

df_user_data_train['last_vs_reg_time_ratio'] = df_user_data_train['last_online_time_seconds']/(1+df_user_data_train['registration_time_seconds'])

df_user_data_train['max_rating_current_ratio'] = df_user_data_train['max_rating']/ (1+df_user_data_train['rating'])
df_user_data_train['current_rating_per_registration_time'] = df_user_data_train['rating']/(1+df_user_data_train['registration_time_days'])
df_user_data_train['Follower_vs_Rating'] = df_user_data_train['follower_count']/(1+df_user_data_train['rating'])
df_user_data_train['Rating_mul_Follower'] = df_user_data_train['rating'] * df_user_data_train['follower_count']
df_user_data_train['follower_vs_Contribution'] = df_user_data_train['follower_count']/(1+df_user_data_train['contribution'])



df_problem_data_train = pd.get_dummies(df_problem_data_train,columns=['level_type'])
df_user_data_train = pd.get_dummies(df_user_data_train,columns=['country','rank'])



In [7]:
df_user_data_train.fillna(0,inplace=True)
df_user_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Columns: 110 entries, user_id to rank_intermediate
dtypes: float64(19), int64(6), object(1), uint8(84)
memory usage: 1018.4+ KB


In [8]:
df_user_data_train.max()

user_id                                    user_999
submission_count                               4570
problem_solved                                 4476
contribution                                    171
follower_count                                10575
last_online_time_seconds                   21357687
max_rating                                  983.085
rating                                      911.124
registration_time_seconds                 219474738
problem_sub_ratio                                 1
last_online_time_minutes                     355961
last_online_time_hours                      5932.69
last_online_time_days                       247.195
last_online_time_months                     8.23985
last_online_time_years                     0.677248
registration_time_minutes               3.65791e+06
registration_time_hours                     60965.2
registration_time_days                      2540.22
registration_time_months                    84.6739
registration

In [9]:
df_user_data_train.head()

,user_id,submission_count,problem_solved,contribution,follower_count,last_online_time_seconds,max_rating,rating,registration_time_seconds,problem_sub_ratio,...,country_Ukraine,country_United Kingdom,country_United States,country_Uzbekistan,country_Venezuela,country_Vietnam,rank_advanced,rank_beginner,rank_expert,rank_intermediate
0,user_3311,47,40,0,4,1483429,348.337,330.849,17549689,0.851064,...,0,0,0,0,0,0,0,0,0,1
1,user_3028,63,52,0,17,6596909,405.677,339.450,42342800,0.825397,...,0,0,0,0,0,0,0,0,0,1
2,user_2268,226,203,8,24,29022,307.339,284.404,29968522,0.898230,...,0,0,0,0,0,0,0,1,0,0
3,user_480,611,490,1,94,337575,525.803,471.330,133515708,0.801964,...,1,0,0,0,0,0,1,0,0,0
4,user_650,504,479,12,4,8981641,548.739,486.525,88675627,0.950397,...,0,0,0,0,0,0,1,0,0,0


In [10]:
df_train  = pd.merge(df_train_submission,df_user_data_train, how='left', on='user_id')
df_train = pd.merge(df_train,df_problem_data_train,how='left',on='problem_id')

df_test  = pd.merge(df_test_sub,df_user_data_train, how='left', on='user_id')
df_test = pd.merge(df_test,df_problem_data_train,how='left',on='problem_id')

X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

X_test = df_test.drop(['ID','user_id','problem_id'],axis=1).values

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)

In [11]:
clf_RF =RandomForestClassifier()
y_pred_cross = cross_val_predict(clf_RF,X,y,cv=8,n_jobs=-1)
print(f1_score(y,y_pred_cross,average="weighted"))

0.443076467483


In [12]:
clf_RF = RandomForestClassifier()
clf_RF.fit(X,y)
y_pred = clf_RF.predict(X_test)

In [13]:
df_sample_sub['ID'] = df_test.ID
df_sample_sub['attempts_range'] = y_pred
df_sample_sub.to_csv("RF_with_Basic__FE_new.csv",index=False)